In [2]:
import numpy as np 
import pandas as pd 
import statsmodels.api as sm 
import statsmodels.formula.api as smf 


In [3]:
# read data
data = pd.read_stata("https://github.com/albarran/TopicsCausalInference/raw/main/data/nsw.dta")

data.head

<bound method NDFrame.head of                   data_id  treat  ...          re75          re78
0    Dehejia-Wahba Sample    1.0  ...      0.000000   9930.045898
1    Dehejia-Wahba Sample    1.0  ...      0.000000   3595.894043
2    Dehejia-Wahba Sample    1.0  ...      0.000000  24909.449219
3    Dehejia-Wahba Sample    1.0  ...      0.000000   7506.145996
4    Dehejia-Wahba Sample    1.0  ...      0.000000    289.789886
..                    ...    ...  ...           ...           ...
440  Dehejia-Wahba Sample    0.0  ...  12357.219727      0.000000
441  Dehejia-Wahba Sample    0.0  ...  13371.250000      0.000000
442  Dehejia-Wahba Sample    0.0  ...  16341.160156  16900.300781
443  Dehejia-Wahba Sample    0.0  ...  16946.630859   7343.963867
444  Dehejia-Wahba Sample    0.0  ...  23031.980469   5448.800781

[445 rows x 11 columns]>

In [4]:
# ------------------------------------------------------------------------------
#  ****************************************************
#  * Checking balancing of observed characteristics
#  ****************************************************

XX = ["age", "educ", "black", "hisp", "marr", "nodegree"]

for X in XX:
  form = X + " ~ treat"
  est = smf.ols(form, data).fit()
  est = est.get_robustcov_results()
  print(X)
  print(est.summary().tables[1])


age
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     25.0538      0.438     57.221      0.000      24.193      25.914
treat          0.7624      0.684      1.114      0.266      -0.582       2.107
educ
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.0885      0.100    100.735      0.000       9.892      10.285
treat          0.2575      0.179      1.442      0.150      -0.093       0.608
black
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8269      0.024     35.166      0.000       0.781       0.873
treat          0.0163      0.036      0.458      0.647      -0.054       0.086
hisp
                 coef    std err

In [5]:
# ****************************************************
# *** ATE
# ****************************************************
est = smf.ols("re78 ~ treat", data).fit()
est = est.get_robustcov_results()
print(est.summary().tables[1])


                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   4554.8011    340.204     13.388      0.000    3886.187    5223.415
treat       1794.3424    670.824      2.675      0.008     475.949    3112.736


In [ ]:
  
# ****************************************************
# *** Including covariates
# ****************************************************

est = smf.ols("re78 ~ treat + age + I(age**2.0) + educ + black + hisp + marr + nodegree", data).fit()
est = est.get_robustcov_results()
print(est.summary().tables[1])


                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      -421.9575   5024.842     -0.084      0.933   -1.03e+04    9453.966
treat          1669.9711    672.485      2.483      0.013     348.256    2991.686
age             178.2931    252.375      0.706      0.480    -317.729     674.316
I(age ** 2.0)    -2.0837      4.023     -0.518      0.605      -9.992       5.824
educ            378.0895    193.513      1.954      0.051      -2.245     758.424
black         -2212.4878    996.721     -2.220      0.027   -4171.462    -253.513
hisp            118.6311   1356.568      0.087      0.930   -2547.595    2784.857
marr             82.7650    934.225      0.089      0.929   -1753.379    1918.909
nodegree       -106.8054   1050.525     -0.102      0.919   -2171.528    1957.917


In [ ]:
# ****************************************************
# *** Treatment heterogeneity
# ****************************************************
  
# ** by education

data['higheduc']  = (data["educ"] > 8) 

est = smf.ols("re78 ~ C(treat)*C(higheduc) + age + I(age**2.0) + educ + black + hisp + marr + nodegree", data).fit()
est = est.get_robustcov_results()
print(est.summary().tables[1])



                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
Intercept                            2312.6430   5099.603      0.453      0.650   -7710.347    1.23e+04
C(treat)[T.1.0]                       -53.6137    961.952     -0.056      0.956   -1944.278    1837.051
C(higheduc)[T.True]                   567.8626   1284.936      0.442      0.659   -1957.609    3093.334
C(treat)[T.1.0]:C(higheduc)[T.True]  2047.6298   1204.737      1.700      0.090    -320.215    4415.474
age                                   179.3123    251.106      0.714      0.476    -314.224     672.848
I(age ** 2.0)                          -2.0682      3.996     -0.518      0.605      -9.921       5.785
educ                                   93.1050    332.550      0.280      0.780    -560.504     746.714
black                               -2226.3435    986.471     -2